<a href="https://colab.research.google.com/github/wiss0212/Super_resolution_3D_Model-/blob/main/TrainingSR3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/wiss0212/Super_resolution_3D_Model-.git

Cloning into 'Super_resolution_3D_Model-'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [ ]:
import argparse
from argparse import ArgumentParser
import glob
import cv2
import PIL
import re
import os, glob, datetime
import numpy as np
import math
import tensorflow as tf
import numpy as np
from scipy.io import loadmat
from skimage.transform import rescale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import  Input,Conv3D, BatchNormalization,Activation,Subtract, Reshape,Lambda
from tensorflow.keras.models import Model, load_model
from tensorflow.python.keras.utils import conv_utils
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD
#import data_generator as dg
import tensorflow.keras.backend as K
import skimage
from skimage.metrics import structural_similarity as ssim
from skimage.transform import rescale, resize
from skimage.io import imread, imsave
from tensorflow.keras.layers import PReLU
from tensorflow.keras.datasets import mnist


## Params
parser = argparse.ArgumentParser(description='Keras DIVA3D')
parser.add_argument('--model', default='DIVA3D', type=str, help='choose a type of model')
parser.add_argument('--kernel_size', default=3, type=int, help='kernel size')
parser.add_argument('--batch_size', default=12, type=int, help='batch size')
parser.add_argument('--train_data', default='/content/drive/MyDrive/Colab Notebooks/DATA_3D/', type=str, help='path of train data')

parser.add_argument('--epoch', default=500, type=int, help='number of train epoches')
parser.add_argument('--lr', default=1e-3, type=float, help='initial learning rate for Adam')
parser.add_argument('--save_every', default=1, type=int, help='save model at every x epoches')
parser.add_argument('-f', '--file', required=False)
args = parser.parse_args()


kernel_size = args.kernel_size
batch_size = args.batch_size

save_dir = os.path.join('/content/drive/MyDrive/Colab Notebooks/models', args.model+'_'
                        + 'CBCT_SR3')   # + str(sigma)

if not os.path.exists(save_dir):
     print("save_dir",save_dir)
     os.mkdir(save_dir)



##--------------------------------------------------------------------------------------------------------
class Hamiltonian_Conv3D(Conv3D):

    def __init__(self, filters, kernel_size, kernel_3=None, kernel_4=None, activation=None, **kwargs):

        self.rank = 7               # Dimension of the kernel
        self.num_filters = filters  # Number of filter in the convolution layer
        self.kernel_size = conv_utils.normalize_tuple(kernel_size, self.rank, 'kernel_size')
        self.kernel_3 = kernel_3    # Weights from original potential
        self.kernel_4 = kernel_4    # Weights from interaction

        super(Hamiltonian_Conv3D, self).__init__(self.num_filters, self.kernel_size,
              activation=activation, use_bias=False, **kwargs)

    def build(self, input_shape):
        if K.image_data_format() == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                     'should be defined. Found `None`.')

        #don't use bias:
        self.bias = None

        #consider the layer built
        self.built = True


        # Define nabla operator
        weights_1 = tf.constant([[[ 3.,-1., 0.],
                                  [-1.,-1., 0.],
                                  [ 0., 0., 0.]],
                                  [[-1.,-1., 0.],
                                  [-1., 6.,-1.],
                                  [ 0.,-1.,-1.]],
                                 [[ 0., 0., 0.],
                                  [ 0.,-1.,-1.],
                                  [ 0.,-1., 3.]]])

        weights_1 = tf.reshape(weights_1 , [3,3,3, 1])
        weights_1 = tf.repeat(weights_1 , repeats=self.num_filters, axis=3)
        #print('kernel shape of weights_1:',weights_1.get_shape())

        # Define Weights for h^2/2m  (size should be same as the nabla operator)
        weights_2 = self.add_weight(shape=weights_1.get_shape(),
                                      initializer= 'Orthogonal',
                                      name='kernel_h^2/2m',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        #print('kernel shape of weights_2:',weights_2.get_shape())


        # Define the Hamiltonian kernel
        self.kernel = weights_1*weights_2 +self.kernel_3 + self.kernel_4
        #print('self.kernel',self.kernel.get_shape())

        self.built = True
        super(Hamiltonian_Conv3D, self).build(input_shape)

    # Do the 3D convolution using the Hamiltonian kernel
    def convolution_op(self, inputs, kernel):
        if self.padding == "causal":
            tf_padding = "VALID"  # Causal padding handled in `call`.
        elif isinstance(self.padding, str):
            tf_padding = self.padding.upper()
        else:
            tf_padding = self.padding


        return tf.nn.convolution(
            inputs,
            kernel,
            strides=list(self.strides),
            padding=tf_padding,
            dilations=list(self.dilation_rate),
            data_format=self._tf_data_format,
            name=self.__class__.__name__,
        )

    def call(self, inputs):
        outputs = self.convolution_op(inputs, self.kernel)
        return outputs




# -------------------------------------------------------------------------------------------------------------------

def DIVA3D(depth,filters=64,image_channels=1, kernel_size= args.kernel_size, use_bnorm=True):
    layer_count = 0
    inpt = Input(shape=(None,None,None,image_channels),name = 'input'+str(layer_count))

    # Get the initial patches /initial_patches
    initial_patches = Conv3D(filters=filters, kernel_size=(kernel_size,kernel_size,kernel_size), strides=(1,1,1),kernel_initializer='Orthogonal', padding='same',name = 'initial_patches')(inpt)
    initial_patches = Activation('relu',name = 'initial_patch_acti')(initial_patches)


    print(initial_patches.get_shape())

    # interaction layer
    inter = Conv3D(filters=filters, kernel_size=(kernel_size,kernel_size,kernel_size), strides=(1,1,1),kernel_initializer='Orthogonal', padding='same',name = 'interactions')(initial_patches)
    inter = Activation('relu',name = 'interaction_acti'+str(layer_count))(inter)

    print(inter.get_shape())


    # Get contributions of the original potential in the Hamiltonian kernel
    ori_poten_kernel = tf.keras.layers.MaxPooling3D (pool_size=(21,21,21), strides=(15,15,15), padding='same', name = 'ori_poten_ker', data_format=None )(initial_patches)
    #print('ori_poten_kernel',ori_poten_kernel.get_shape())

    # Get contributions of the interactions in the Hamiltonian kernel
    inter_kernel = tf.keras.layers.MaxPooling3D (pool_size=(21,21,21), strides=(15,15,15), padding='same', name = 'inter_ker', data_format=None )(inter)
    #print('inter_kernel',inter_kernel.get_shape())


    # Get projection coefficients of the initial patches on the Hamiltonian kernel
    x = Hamiltonian_Conv3D(filters=filters, kernel_size=(kernel_size,kernel_size,kernel_size), kernel_3 = ori_poten_kernel, kernel_4 = inter_kernel, strides=(1,1,1), activation='relu',
                              kernel_initializer='Orthogonal', padding='same', name = 'proj_coef')(initial_patches)

    #print('coef',x.get_shape())


    # Do Thresholding (depth depends on the noise intensity)
    for i in range(depth+10):
      layer_count += 1
      x = Conv3D(filters=filters, kernel_size=(kernel_size,kernel_size,kernel_size), strides=(1,1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)

      layer_count += 1
      x = BatchNormalization(axis=4, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
        #x = BatchNormalization(axis=4, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)

      # Thresholding
    x = Activation('relu',name = 'Thresholding'+str(layer_count))(x)


    # Inverse projection
    x = Conv3D(filters=image_channels, kernel_size=(kernel_size,kernel_size,kernel_size), strides=(1,1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'inv_trans')(x)

    x = Subtract(name = 'subtract')([inpt, x])



    model = Model(inputs=inpt, outputs=x)

    return model

##----------------------------------------------------------------------------------------------------------------------


def findLastCheckpoint(save_dir):
    file_list = glob.glob(os.path.join(save_dir,'model_*.hdf5'))  # get name list of all .hdf5 files
    #file_list = os.listdir(save_dir)
    if file_list:
        epochs_exist = []
        for file_ in file_list:
            result = re.findall(".*model_(.*).hdf5.*",file_)
            #print(result[0])
            epochs_exist.append(int(result[0]))
        initial_epoch=max(epochs_exist)
    else:
        initial_epoch = 0
    return initial_epoch


def show(x,title=None,cbar=False,figsize=None):
    import matplotlib.pyplot as plt
    plt.figure(figsize=figsize)
    plt.imshow(x,interpolation='nearest',cmap='gray')
    if title:
        plt.title(title)
    if cbar:
        plt.colorbar()
    plt.show()


def log(*args,**kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"),*args,**kwargs)

def lr_schedule(epoch):
    initial_lr = args.lr
    if epoch<=20:
        lr = initial_lr
    elif epoch<=35:
        lr = initial_lr/10
    elif epoch<=50:
        lr = initial_lr/20
    elif epoch<=60:
        lr = initial_lr/50
    elif epoch<=70:
        lr = initial_lr/100
    elif epoch<=80:
        lr = initial_lr/200
    else:
        lr = initial_lr/20
    log('current learning rate is %2.8f' %lr)
    return lr

def train_datagen(epoch_iter=500,epoch_num=5,batch_size=64,data_dir=args.train_data):
  import random
  import math

  while(True):
      n_count = 0
      if n_count == 0:
          #print(n_count)
          xs, ys = datagenerator(data_dir)
          assert len(xs)%args.batch_size ==0, \
          log('make sure the last iteration has a full batchsize, this is important if you use batch normalization!')
          xs = xs.astype('float32')
          ys = ys.astype('float32')

          indices = list(range(xs.shape[0]))
          n_count = 1
      for _ in range(epoch_num):
          np.random.shuffle(indices)    # shuffle
          for i in range(0, len(indices), batch_size):
              batch_x = xs[indices[i:i+batch_size]]
              batch_y = ys[indices[i:i+batch_size]]


              yield batch_y, batch_x


##--------------------------------------------------------------------------------------------------------------------------

##--------------------------------------------------------------------------------------------------------------------------

patch_size, stride = 60, 1
aug_times = 1
num_noise_realiza = 1
scales = [1] #, 0.9, 0.8, 0.7
batch_size = args.batch_size



def data_aug(img, mode=0):

    if mode == 0:
        return img
    elif mode == 1:
        return np.flipud(img)
    elif mode == 2:
        return np.rot90(img)
    elif mode == 3:
        return np.flipud(np.rot90(img))
    elif mode == 4:
        return np.rot90(img, k=2)
    elif mode == 5:
        return np.flipud(np.rot90(img, k=2))
    elif mode == 6:
        return np.rot90(img, k=3)
    elif mode == 7:
        return np.flipud(np.rot90(img, k=3))






def gen_patches(file_name):

    # read image
    #img = cv2.imread(file_name, 0)  # gray scale
    #print("file_name:",file_name)
    I = loadmat(file_name)          # the CT gray scale image

    clean_img = I['img']            # the CT data


    img= I['img_cbct']
    h, w , d = img.shape


    # For SR only using DIVA3D-A

    img = img[0:h:2,0:w:2,0:d:2]
    img = rescale(img, 2, anti_aliasing=False)
   # print(img.shape)


    patches = []
    clean_patches = []

    # data augmentation with different rotation
    for k in range(0, aug_times):
        mode_k=np.random.randint(0,8)
        x_aug = data_aug(img, mode=mode_k)
        clean_x_aug = data_aug(clean_img, mode=mode_k)

        patches.append(x_aug)
        clean_patches.append(clean_x_aug)

    return clean_patches, patches



def datagenerator(data_dir=args.train_data,verbose=False):

    #file_list = glob.glob(data_dir+'/*.png')  # get name list of all .png files
    file_list = glob.glob(data_dir+'/*.mat')  # get name list of all .mat files

    # initrialize
    data = []
    data_clean = []

    # generate patches
    for i in range(len(file_list)):




            clean_patch, patch = gen_patches(file_list[i])

            data.append(patch)
            data_clean.append(clean_patch)

            #if verbose:
            #   print(str(i+1)+'/'+ str(len(file_list)) + ' is done ^_^')
    data = np.array(data, dtype=np.float32)
    print("good dim =", data.shape)
    # data_clean = np.array(data_clean, dtype=np.float32)

    # # do for cbct data
    data = np.array(data, dtype='float32')
    data = data.reshape((data.shape[0]*data.shape[1],data.shape[2],data.shape[3],data.shape[4],1))
    discard_n = len(data)-len(data)//batch_size*batch_size;
    #data = np.delete(data,range(discard_n),axis = 0)
    # #data=tf.image.adjust_contrast(data, contrast_factor=2)
    # # do for Mct  data
    data_clean = np.array(data_clean, dtype='float32')
    data_clean = data_clean.reshape((data_clean.shape[0]*data_clean.shape[1],data_clean.shape[2],data_clean.shape[3],data_clean.shape[4],1))
    discard_n = len(data_clean)-len(data_clean)//batch_size*batch_size;
    #data_clean = np.delete(data_clean,range(discard_n),axis = 0)
    #data_clean=tf.image.adjust_contrast(data_clean, contrast_factor=2)
    print('^_^-training data finished-^_^')
    print("bad dim =", data.shape)
    print(data_clean.shape)

    return data_clean, data


##---------------------------------------------------------------------------------------------------------


# define loss
def sum_squared_error(y_true, y_pred):
    #return K.mean(K.square(y_pred - y_true), axis=-1)
    #return K.sum(K.square(y_pred - y_true), axis=-1)/2
    return K.sum(K.square(y_pred - y_true))/2

# Loss function ssim
#def ssim_loss(y_true, y_pred):
    #return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))

if __name__ == '__main__':
    # model selection
    model = DIVA3D(depth=20,filters=64,image_channels=1,use_bnorm=True)
    model.summary()

    # load the last model in matconvnet style
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    if initial_epoch > 0:
        print('resuming by loading epoch %03d'%initial_epoch)
        model = DIVA3D(depth=20,filters=64,image_channels=1,use_bnorm=True)
       # model = load_model(os.path.join(save_dir,'model_%03d.hdf5'%initial_epoch), compile=False)
        model.load_weights(os.path.join(save_dir,'model_%03d.hdf5'%initial_epoch))
        log('load trained model architecture')

    # compile the model
    model.compile(optimizer=Adam(0.001),
                  loss= tf.keras.losses.MeanSquaredError(), #tf.keras.losses.CosineSimilarity (axis=-1, reduction="auto", name="cosine_similarity"), loss=ssim_loss,
                  #metrics=[ssim_loss, 'accuracy'])

                  metrics=[tf.keras.metrics.MeanSquaredError(),
                           tf.keras.metrics.RootMeanSquaredError(),
                           tf.keras.metrics.MeanSquaredLogarithmicError(),
                           tf.keras.metrics.MeanAbsoluteError(),
                           sum_squared_error])

    # tf.keras.metrics.MeanAbsolutePercentageError(), tf.keras.metrics.CosineSimilarity(name="cosine_similarity", dtype=None, axis=-1),
    # tf.keras.metrics.LogCoshError(),

    # use call back functions
    checkpointer = ModelCheckpoint(os.path.join(save_dir,'model_{epoch:03d}.hdf5'),
                verbose=1, save_weights_only=False, period=10)
    csv_logger = CSVLogger(os.path.join(save_dir,'log.csv'), append=True, separator=',')
    lr_scheduler = LearningRateScheduler(lr_schedule)

    history = model.fit(train_datagen(batch_size=args.batch_size),
                steps_per_epoch=700, epochs=args.epoch, verbose=1, initial_epoch=initial_epoch,
                callbacks=[checkpointer,csv_logger,lr_scheduler])


##---------------------------------------------------------------------------------------------------------


    # plot loss functions
    # list all data in history
    #print(history.history.keys())
    # summarize history for loss
   # plt.plot((history.history['loss']))
    #plt.title('model loss')
   # plt.ylabel('loss')
    #plt.xlabel('epoch')
   # plt.legend(['train'], loc='upper right')
   # plt.show()


(None, None, None, None, 64)
(None, None, None, None, 64)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input0 (InputLayer)            [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 initial_patches (Conv3D)       (None, None, None,   1792        ['input0[0][0]']                 
                                None, 64)                                                         
                                                                                                  
 initial_patch_acti (Activation  (None, None, None,   0          ['initial_patches[0][0]']        
 )                              None

2023-05-23 17:11:28: load trained model architecture
good dim = (200, 1, 60, 60, 60)
^_^-training data finished-^_^
bad dim = (200, 60, 60, 60, 1)
(200, 60, 60, 60, 1)
2023-05-23 17:12:01: current learning rate is 0.00000050
Epoch 51/100


50/50 [==============================] - 111s 2s/step - loss: 0.0100 - mean_squared_error: 0.0100 - root_mean_squared_error: 0.1001 - mean_squared_logarithmic_error: 0.0044 - mean_absolute_error: 0.0633 - sum_squared_error: 4327.7173 - lr: 5.0000e-07
2023-05-23 17:13:52: current learning rate is 0.00000020
Epoch 52/100
50/50 [==============================] - 82s 2s/step - loss: 0.0101 - mean_squared_error: 0.0101 - root_mean_squared_error: 0.1003 - mean_squared_logarithmic_error: 0.0044 - mean_absolute_error: 0.0633 - sum_squared_error: 4344.3232 - lr: 2.0000e-07
2023-05-23 17:15:14: current learning rate is 0.00000020
Epoch 53/100
50/50 [==============================] - 82s 2s/step - loss: 0.0102 - mean_squared_error: 0.0102 - root_mean_squared_error: 0.1012 - mean_squared_logarithmic_error: 0.0044 - mean_absolute_error: 0.0644 - sum_squared_error: 4422.4902 - lr: 2.0000e-07
2023-05-23 17:16:37: current learning rate is 0.00000020
Epoch 54/100
50/50 [==============================] 